# Titanic: Top 5% on Leaderboard with SVM

**By: Brian Rafferty**

Welcome to my "Titanic: Machine Learning from Disaster" Jupyter Notebook! This notebook will highlight how I tackled the problem, and can act as a general blueprint to undertake most Data Science projects.

In [ ]:
#data analysis and wrangling
import pandas as pd
import numpy as np

#visualization
import seaborn as sns
import matplotlib.pyplot as plt

#machine learning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#access data and create pandas dataframes
train_df = pd.read_csv('../input/titanic/train.csv')
test_df = pd.read_csv('../input/titanic/test.csv')
combine = [train_df, test_df]

In [ ]:
#see all of the types of data contained (print every column)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(train_df.head())

In [ ]:
#determine datatypes of the training data and see which have nulls
train_df.info()

In [ ]:
#determine datatypes of the testing data and see which have nulls
test_df.info()

# Before moving forward, look at the data and answer these questions:

**Which features are categorical?**
     - Survived, Sex, and Embarked
     - Pclass is ordinal
**Which features are numerical?**
     - Age and Fare are continuous
     - SibSp and Parch are discrete
**Which features are mixed data types?**
     - Ticket and Cabin
**Do any features possibly contain errors? If so, which ones?**
     - Yes, Name has ambiguous data listed
**Which features are missing values?**
     - Cabin, Age, and Embarked

In [ ]:
#see distribution statistics for numerical values
train_df[['Age','Fare','SibSp','Parch']].describe()

In [ ]:
#see distribution statistics for categorical values
train_df.describe(include=['O'])

In [ ]:
#analyze the features by pairing them together 'pivoting features'
#Note: only do this with categorical, ordinal, or discrete variables

#strong correlation between being Pclass: 3 and not surviving
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
#strong correlation between being Sex: female and Survived
train_df[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
#SibSp 5 & 8 have zero correlation with Survived, while other groups do, indicating a
#beneficial opportunity to utilize 'feature engineering' (deriving a new feature or set 
#of features from this one)
train_df[['SibSp', 'Survived']].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
#same scenario with the feature Parch, will also conduct feature engineering here
train_df[['Parch', 'Survived']].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
#analyze the numerical features now (charts are useful for these)
g = sns.FacetGrid(train_df, col='Survived')
g.map(plt.hist, 'Age', bins=20)

In [ ]:
#can plot comparisons between continuous and categorical if the categorical is numeric
g = sns.FacetGrid(train_df, col='Survived', row='Pclass', height=2.2, aspect=1.6)
g.map(plt.hist, 'Age', alpha=0.5, bins=20)
g.add_legend()

In [ ]:
#plot the categorical features that indicated highest correlation, compare based upon where they embarked
g = sns.FacetGrid(train_df, row='Embarked', height=2.2, aspect=1.6)
g.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep')
g.add_legend()

In [ ]:
#plot the categorical features that indicated highest correlation, compare 
#based upon where they embarked and their fare price
g = sns.FacetGrid(train_df, row='Embarked', col='Survived', height=2.2, aspect=1.6)
g.map(sns.barplot, 'Sex', 'Fare', alpha=0.5, ci=None)
g.add_legend()

In [ ]:
#Wrangle the data

#Correcting
#start with correcting the data by dropping features that have no use
print('Before', train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]

print('After', train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

In [ ]:
#Creating
#create new feature by extracting from existing
#pull title out of name feature
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train_df['Title'], train_df['Sex'])

In [ ]:
#wrangle the new Title feature, compress every row with low counts into 1 row
#then compress variations into same row (Mlle == Miss, Ms == Miss, Mme == Mrs)
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

In [ ]:
#compare the engineered feature Title with Survived to see correlation
train_df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [ ]:
#convert the categorical titles to ordinal for future use in the ML model
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_df[['Name', 'Title']].head()

In [ ]:
#now drop Name and PassengerId feature, since they are no 
#longer needed in the training set
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]

In [ ]:
#convert the categorical feature Sex to numerical values 
#for future use in the ML model
sex_mapping = {'male': 0, 'female': 1}

for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map(sex_mapping)

train_df[['Title', 'Sex']].head()

In [ ]:
#Completing
#now is time to complete a numerical continuous feature with 
#missing or null values
#start with Age
#guess ages by using the median values across Pclass and gender (Age 
#for Pclass=1 and Gender=0, Pclass=1 and Gender=1, ...)
grid = sns.FacetGrid(train_df, row='Pclass', col='Sex', height=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=0.5, bins=20)
grid.add_legend()

In [ ]:
#start with an empty array to contain the guessed ages
guess_ages = np.zeros((2,3))

#now iterate over Sex (0,1) and Pclass(1,2,3) to calc the guessed ages
for dataset in combine:
    for i in range(2):
        for j in range(3):
            guess_df = dataset[(dataset['Sex'] == i) & (dataset['Pclass'] == j+1)]['Age'].dropna()
            
            age_guess = guess_df.median()
            
            #convert random age float to nearest .5 age
            guess_ages[i,j] = int(age_guess/0.5 + 0.5) * 0.5
            

    for i in range(2):
        for j in range(3):
            dataset.loc[(dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1), 'Age'] = guess_ages[i,j]
            
    dataset['Age'] = dataset['Age'].astype(int)

train_df.head()

In [ ]:
#create Age bands and determine correlations with Survived
train_df['AgeBand'] = pd.cut(train_df['Age'], 7)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

In [ ]:
#replace Age with ordinal values based upon these bands
for dataset in combine:
    dataset.loc[dataset['Age'] <= 11.429, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 11.429) & (dataset['Age'] <= 22.857), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 22.857) & (dataset['Age'] <= 34.286), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 34.286) & (dataset['Age'] <= 45.714), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 45.714) & (dataset['Age'] <= 57.143), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 57.143) & (dataset['Age'] <= 68.571), 'Age'] = 5
    dataset.loc[dataset['Age'] > 68.571, 'Age'] = 6
    dataset['Age'] = dataset['Age'].astype(int)


train_df[['Age', 'AgeBand']].head()

In [ ]:
#drop AgeBand, now that ages are placed in correct bands
train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]

train_df.head()

In [ ]:
#Creating
#create new feature called FamilySize which combines Parch and 
#SibSp, this will allow us to drop 2 columns and replace it with 1
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

#No correlation between FamilySize 8 & 11 and Survived, must engineer new feature
train_df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
#Make a new feature called IsAlone to eliminate values of zero
for dataset in combine:
    dataset['IsAlone'] = 0
    #change value to 1 if family size is 1 person
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

#Notice correlation between not being alone and surviving
train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

In [ ]:
#drop Parch, SibSp, and FamilySize and keep IsAlone
train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]

train_df.head()

In [ ]:
#create artificial feature combining Pclass and Age
for dataset in combine:
   dataset['Age*Class'] = dataset.Age * dataset.Pclass

train_df[['Age*Class', 'Survived']].groupby(['Age*Class'], as_index=False).mean()

In [ ]:
#create artificial feature combining Title and Class
for dataset in combine:
   dataset['Title*Class'] = dataset.Title * dataset.Pclass
    
train_df[['Title*Class', 'Survived']].groupby(['Title*Class'], as_index=False).mean()

In [ ]:
#Completing
#complete the embarked feature by find the mode, and 
#filling that value in all the null spots
freq_port = train_df.Embarked.dropna().mode()[0]

for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)

#Correlation between Embarked: C and Survived
train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
#map Embarked to numeric values for the ML model
port_mapping = {'S': 0, 'C': 1, 'Q': 2}

for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map(port_mapping)

train_df.head()

In [ ]:
#Completing
#fill the missing fare values by finding the mode and 
#replacing the nulls with it
freq_fare = train_df.Fare.dropna().mode()[0]

for dataset in combine:
    dataset['Fare'] = dataset['Fare'].fillna(freq_fare)

In [ ]:
#create new feature called FareBand, just like AgeBand before
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

In [ ]:
#convert Fare into ordinal values based upon results
for dataset in combine:
    dataset.loc[dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31.0), 'Fare'] = 2
    dataset.loc[dataset['Fare'] > 31.0, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

#remove FareBand feature
train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]

train_df.head()

In [ ]:
#check test values too, make sure columns match up
test_df.head()

In [ ]:
#Modeling
#time to train a model and predict a solution
X = train_df.drop('Survived', axis=1)
y = train_df['Survived']
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.5, random_state=1)
X_test = test_df.drop('PassengerId', axis=1).copy()

In [ ]:
#Model Selection
#this is a linear problem, can we figure out a 
#function f(x) = y that predicts survivability?
#spot check Linear ML algorithms to determine the 
#most accurate one
#notice they are all close, but SVM is highest
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('SVM', SVC(gamma='auto')))
models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('PERC', Perceptron()))
models.append(('LSVC', LinearSVC()))
models.append(('SGD', SGDClassifier()))
models.append(('RF', RandomForestClassifier()))

for name, model in models:
    model.fit(X_train, Y_train)
    score = model.score(X_train, Y_train)
    print('Accuracy of {} on training set: {}'.format(name, score))

    testScore = model.score(X_validation, Y_validation)
    print('Accuracy of {} on testing set: {}'.format(name, testScore))

In [ ]:
#tune hyperparameters for SVM 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
model = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
model.fit(X_train, Y_train)
hyperparams = model.best_params_
print(hyperparams)

In [ ]:
#best params: {'C': 100, 'gamma': 0.01, kernel': 'rbf'}
#Original Train: 0.8674157303370786 
#Tuned Train: 0.8674157303370786
#Original Validation: 0.7959641255605381 
#Tuned Validation: 0.7959641255605381
#normally tuning hyperparameters will yield positive 
#results, but in this case it made no difference
tuned_model = SVC(C=100, gamma=0.01, kernel='rbf')
tuned_model.fit(X_train, Y_train)
train_score = tuned_model.score(X_train, Y_train)
test_score = tuned_model.score(X_validation, Y_validation)
print("Train: {}\nValidation: {}".format(train_score, test_score))

In [ ]:
#make predictions with model and save output data
predictions = model.predict(X_test)

submission = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived': predictions})

print('Printing Submission CSV')

submission.to_csv('./submission.csv', index=False)

# Final score with test.csv outputs 0.79665

- Teams with this score start appearing at position 785. With around 17,000 total teams, that means this analysis and model results in the **top 5%** of all submissions! 

- Success aside, there is still much room to improve my skillset with feature engineering, which is the most likely candidate holding the model back from better predictions.